In [1]:
#Paulo Palomino Vilcapoma     20155714

# Visualización

*Adaptado de https://www.kaggle.com/gpreda/plotly-tutorial-120-years-of-olympic-games*

## **Agenda**


**Analysis**

- Introduction 
- The data    
- Games and venues
- Sports
- Countries
- Athlets
- Medals 
- References 
- Known issues

**Plotly chart types and functions**

- Scatter
- append_trace
- Bar
- create_table
- Box
- Heatmap
- Pie
- Choropleth
- create_distplot
- Slider (animation)


## **Introducción**

Aprenderemos sobre algunas de las herramientas de visualización más importantes dentro del ecosistema de librerías de Python: Plotly y Dash

![very good|512x397, 20%](https://www.pngkit.com/png/full/861-8618685_numfocus-plotly-dash-logo.png)





### [Plotly for Python](https://plotly.com/python/): 
Es una biblioteca de gráficos para Python. Permite crear gráficos interactivos con calidad de publicación. Por ejemplo line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.

### [Dash](https://dash.plotly.com/):

Es un framework productivo de Python para el desarrollo de aplicaciones web. 
Es ideal para crear aplicaciones de visualización de datos con interfaces de usuario altamente personalizadas en Python puro. Es especialmente adecuado para cualquier persona que trabaje con datos en Python.

## **Dataset de los Juegos Olímpicos**

Este es un conjunto de datos históricos sobre los Juegos Olímpicos modernos, incluidos todos los Juegos desde Atenas 1896 hasta Río 2016. (Fuente: www.sports-reference.com)

El archivo deportista_eventos.csv contiene 271,116 filas y 15 columnas. Cada fila corresponde a un atleta individual que compite en un evento olímpico individual (eventos de atleta). Las columnas son:

-  ID: número único para cada atleta </br>
-  Nombre: nombre del atleta </br>
-  Sexo - M o F </br>
-  Edad - Entero </br>
-  Altura - en centímetros </br>
-  Peso: en kilogramos </br>
-  Equipo - Nombre del equipo </br>
-  NOC - Código de 3 letras del Comité Olímpico Nacional </br>
-  Juegos - Año y temporada </br>
-  Año: entero </br>
-  Temporada: verano o invierno </br>
-  Ciudad - Ciudad anfitriona </br>
-  Deporte - deporte </br>
-  Evento - Evento </br>
-  Medalla: oro, plata, bronce o NA </br>

### Load packages


In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

IS_LOCAL = False
import os

### Lectura de datos


In [2]:
athlete_events_df = pd.read_csv("athlete_events.csv")
noc_regions_df = pd.read_csv("noc_regions.csv")

## Revisión de los datos


Revisemos el tamaño de ambos df

In [3]:
print("Athletes and Events data -  rows:",athlete_events_df.shape[0]," columns:", athlete_events_df.shape[1])
print("NOC Regions data -  rows:",noc_regions_df.shape[0]," columns:", noc_regions_df.shape[1])

Athletes and Events data -  rows: 271116  columns: 15
NOC Regions data -  rows: 230  columns: 3


Veamos los primeros registros

In [4]:
athlete_events_df.head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [7]:
noc_regions_df.head(5)

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


Revisamos los missings

In [5]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(athlete_events_df)

,Total,Percent
Medal,231333,85.326207
Weight,62875,23.191180
Height,60171,22.193821
Age,9474,3.494445
Event,0,0.000000
Sport,0,0.000000
City,0,0.000000
Season,0,0.000000
Year,0,0.000000
Games,0,0.000000


Solo una parte de los atletas tiene medallas, que es lo esperado. Al mismo tiempo, faltan algunas medidas corporales (peso y altura) y la edad (3.5%).

In [6]:
missing_data(noc_regions_df)

,Total,Percent
notes,209,90.869565
region,3,1.304348
NOC,0,0.000000


Falta la mayoría de las notas (90%) pero también faltan algunos de los nombres de las regiones.

## Análisis de lugares y eventos

Veamos en qué años tuvimos los eventos olímpicos y cuáles fueron las sedes.

Primero, revisemos los años de los lugares y la temporada. Tenemos los Juegos Olímpicos de Invierno y Verano. Agrupamos por `Year` y seleccionamos `Season` y obtendremos el número de atletas por evento. Usamos un diagrama de dispersión (scatter plot) para esto.

In [7]:
tmp = athlete_events_df.groupby(['Year', 'City'])['Season'].value_counts()
df = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()

In [8]:
df.head(3)

,Year,City,Season,Athlets
0,1896,Athina,Summer,380
1,1900,Paris,Summer,1936
2,1904,St. Louis,Summer,1301


### Scatter

Se setean los siguientes atributos para el objeto `trace`:  

* x - the points coordinates on x axis;  
* y - the points coordinates on y axis;  
* name - the name associated with the sequence (x,y); 
* marker - the marker used for the border of map areas specified in locations; 
* mode - the representation mode of the scatter graph; here we will use `markers` but frequent used are as well `lines` or `markers+lines`;  

Se pueden manejar varios objetos `trace`; estos serían agregados a la lista `data` que luego será mostrada a través del objetgo `fig`, mediante el objeto `layout`. Para el `layout`, los siguientes atributos son seteados:  
* title - title displayed for the chart;
* xaxis - title and attributes of the title displayed on the x axis;
* yaxes - title and attributes of the title displayed on the y axis;
* hovermode - specify how will be displayed the popups when hover above the points - all popups or only over on the current trace;

In [9]:
trace = go.Scatter(
    x = df['Year'],
    y = df['Athlets'],
    name="Athlets per Olympic game",
    marker=dict(
        color="Blue",
    ),
    mode = "markers"
)
data = [trace]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets1')


En el eje `x` tenemos los años de los juegos olímpicos y en el eje `y` tenemos el número de atletas por juego.

Sobre el título tenemos varios controles que nos permiten controlar el trace. Podemos hacer zoom, desplazar, seleccionar una ventana para hacer zoom, usar la selección de lazo, acercar, alejar, restablecer el zoom.

Además, cuando pasamos el cursor sobre el gráfico, el valor se muestra en una pequeña ventana emergente sobre el punto más cercano.

Observamos que hay años en los que tenemos dos eventos (verano e invierno). De hecho, en la historia de los Juegos Olímpicos, los Juegos comenzaron con solo eventos de verano, luego hubo en el mismo año durante un tiempo juegos de Verano e Invierno y luego, en un momento determinado, comenzaron a programar en diferentes años Verano y juegos de invierno.

Tracemos nuevamente el scatter pero mostrando ahora los juegos de 'Verano' e 'Invierno' con diferentes colores, en el mismo diagrama. Para esto, crearemos dos `traces` diferentes.

También agregaremos líneas al diagrama de dispersión, agregando `lines` al atributo `mode`.

In [10]:
dfS = df[df['Season']=='Summer']; dfW = df[df['Season']=='Winter']

traceS = go.Scatter(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
    marker=dict(color="Red"),
    mode = "markers+lines"
)
traceW = go.Scatter(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(color="Blue"),
    mode = "markers+lines"
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets2')

Podemos observar que desde 1896 hasta 1920 solo hubo juegos de verano. De 1924 a 1992 hubo eventos de 'Verano' e 'Invierno' cada 4 años, con la interrupción debida a la Segunda Guerra Mundial entre 1936 y 1948.

El diagrama de dispersión permite ver los patrones de presencia de los juegos.

Un evento notable que se puede observar es que en `1956` los Juegos Olímpicos de Verano se llevaron a cabo en 2 ciudades diferentes, `Melbourne` y` Estocolmo`. Lo que sucedió fue que debido a las estrictas regulaciones de cuarentena de Australia, los caballos no podían ser admitidos en el país y, por lo tanto, las competiciones ecuestres se llevaban a cabo cuatro meses antes en Estocolmo.

Otro ejemplo, hubo una caída drástica en la presencia en `1980` cuando West Block boicoteó los juegos olímpicos de Moscú. 

Vamos a mostrar cómo podemos crear subtraces con Plotly. Mostraremos la trama anterior de lado a lado, en dos columnas.

In [11]:
traceS = go.Scatter(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
    marker=dict(color="Red"),
    mode = "markers+lines",
    text=dfS['City'],
)
traceW = go.Scatter(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(color="Blue"),
    mode = "markers+lines",
    text=dfW['City']
)

data = [traceS, traceW]

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Number athlets: Summer Games', 'Number athlets: Winter Games'))
fig.append_trace(traceS, 1, 1)
fig.append_trace(traceW, 1, 2)

iplot(fig, filename='events-athlets2')

C:\Users\PERSONAL\Anaconda3\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



### Bar

Vamos a mostrar el número de atletas por juego olímpico usando el "diagrama de barras".

También prepararemos el conjunto de datos para la visualización agregando el nombre de la ciudad.

In [12]:
tmp = athlete_events_df.groupby('Year')['City'].value_counts()
df2 = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()
df2 = df2.merge(df)

También vamos a mostrar cómo podemos mostrar tablas con Plotly.

In [13]:
iplot(ff.create_table(df2.head(3)), filename='jupyter-table2')

In [14]:
dfS = df2[df2['Season']=='Summer']; dfW = df2[df2['Season']=='Winter']

traceS = go.Bar(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
    marker=dict(color="Red"),
    text=dfS['City']
)
traceW = go.Bar(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(color="Blue"),
    text=dfS['City']
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets3')

Con esta vista es más fácil ver cómo los eventos olímpicos se programan juntos desde 1924 y se separan a partir de 1999.

En el siguiente gráfico jugamos un poco más con las opciones de visualización para `go.Bar`.

Utilizaremos las opciones para definir el color, la opacidad, los márgenes para cada diagrama de barras.

Además, cambiamos el diseño para mostrar las barras apiladas (para ver también el número total de atletas por Juego, donde hay dos juegos diferentes cada año).

In [15]:
traceS = go.Bar(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='black',
                    width=0.75),
                opacity=0.7,
            ),
    text=dfS['City'],
    
)
traceW = go.Bar(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(
                color='rgb(11,23,245)',
                line=dict(
                    color='black',
                    width=0.75),
                opacity=0.7,
            ),
    text=dfS['City']
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest',
          barmode='stack'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets4')

### Box

Mostremos la distribución del número de atletas durante las ediciones de los Juegos Olímpicos, agrupados por `Temporada`.

Cambiamos el diseño para rotar el eje y para que sea más fácil leer las etiquetas de las 'Estaciones'.

In [16]:
traceS = go.Box(
    x = dfS['Athlets'],
    name="Summer Games",
    
     marker=dict(
                color='rgba(238,23,11,0.5)',
                line=dict(
                    color='red',
                    width=1.2),
            ),
    text=dfS['City'],
    orientation='h',
    
)
traceW = go.Box(
    x = dfW['Athlets'],
    name="Winter Games",
    marker=dict(
                color='rgba(11,23,245,0.5)',
                line=dict(
                    color='blue',
                    width=1.2),
            ),
    text=dfS['City'],  orientation='h',
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Number of athlets',showticklabels=True),
          yaxis = dict(title = 'Season', showticklabels=True, tickangle=-90), 
          hovermode = 'closest',
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets5')

La gráfica del tipo `go.Box` muestra el mínimo, máximo, primer cuartil y tercer cuartil de la distribución de datos, en este caso, el número de atletas agrupados por temporada.

## Sports

Mostremos la información sobre los deportes usando las funciones que ya exploramos `go.Scatter`,` go.Bar` y `go.Box`.

Primero, contemos cuántos deportes diferentes se jugaron en cada Olimpiada. Usaremos `go.Scatter` para representar el número de deportes por edición.

In [17]:
tmp = athlete_events_df.groupby(['Year', 'City','Season'])['Sport'].nunique()
df = pd.DataFrame(data={'Sports': tmp.values}, index=tmp.index).reset_index()

In [18]:
df.head(3)

,Year,City,Season,Sports
0,1896,Athina,Summer,9
1,1900,Paris,Summer,20
2,1904,St. Louis,Summer,18


In [19]:
dfS = df[df['Season']=='Summer']; dfW = df[df['Season']=='Winter']

traceS = go.Bar(
    x = dfS['Year'],y = dfS['Sports'],
    name="Summer Games",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='red',
                    width=1),
                opacity=0.5,
            ),
    text= dfS['City'],
)
traceW = go.Bar(
    x = dfW['Year'],y = dfW['Sports'],
    name="Winter Games",
    marker=dict(
                color='rgb(11,23,245)',
                line=dict(
                    color='blue',
                    width=1),
                opacity=0.5,
            ),
    text=dfS['City']
)

data = [traceS, traceW]
layout = dict(title = 'Sports per Olympic edition',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of sports'),
          hovermode = 'closest',
          barmode='stack'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')

Mostremos ahora el número de atletas por deporte para cada año.

Para cada deporte, cada año, se trazará un punto.

In [20]:
tmp = athlete_events_df.groupby(['Year', 'City','Season'])['Sport'].value_counts()
df = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()
df.head()

,Year,City,Season,Sport,Athlets
0,1896,Athina,Summer,Athletics,106
1,1896,Athina,Summer,Gymnastics,97
2,1896,Athina,Summer,Shooting,65
3,1896,Athina,Summer,Cycling,41
4,1896,Athina,Summer,Tennis,23


In [21]:
dfS = df[df['Season']=='Summer']; dfW = df[df['Season']=='Winter']


traceS = go.Scatter(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='red',
                    width=1),
                opacity=0.5,
            ),
    text= "City:"+dfS['City']+" Sport:"+dfS['Sport'],
    mode = "markers"
)
traceW = go.Scatter(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(
                color='rgb(11,23,245)',
                line=dict(
                    color='blue',
                    width=1),
                opacity=0.5,
            ),
   text= "City:"+dfW['City']+" Sport:"+dfW['Sport'],
    mode = "markers"
)

data = [traceS, traceW]
layout = dict(title = 'Number of athlets per sport for each Olympic edition',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of athlets per sport'),
          hovermode='closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')

La leyenda muestra el deporte y la ciudad, así como la cantidad de atletas por deporte por edición.

Mostremos también la distribución del número de atletas por deporte. Para esto, agrupamos por 'Año' y 'Temporada' y contamos los atletas por cada deporte.

In [22]:
tmp = athlete_events_df.groupby(['Year', 'City','Season'])['Sport'].value_counts()
df = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()
df.head(3)

,Year,City,Season,Sport,Athlets
0,1896,Athina,Summer,Athletics,106
1,1896,Athina,Summer,Gymnastics,97
2,1896,Athina,Summer,Shooting,65


Definamos una lista con todos los deportes.

In [23]:
sports = (athlete_events_df.groupby(['Sport'])['Sport'].nunique()).index

Crearemos una función para mostrar `trace` y una función para mostrar el conjunto de trazas.

También filtraremos los juegos por verano e invierno.

In [24]:
def draw_trace(dataset, sport):
    dfS = dataset[dataset['Sport']==sport];
    trace = go.Box(
        x = dfS['Athlets'],
        name=sport,
         marker=dict(
                    line=dict(
                        color='black',
                        width=0.8),
                ),
        text=dfS['City'], 
        orientation = 'h'
    )
    return trace


def draw_group(dataset, title,height=800):
    data = list()
    for sport in sports:
        data.append(draw_trace(dataset, sport))


    layout = dict(title = title,
              xaxis = dict(title = 'Number of athlets',showticklabels=True),
              yaxis = dict(title = 'Sport', showticklabels=True, tickfont=dict(
                family='Old Standard TT, serif',
                size=8,
                color='black'),), 
              hovermode = 'closest',
              showlegend=False,
                  width=800,
                  height=height,
             )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='events-sports1')

# select only Summer Olympics
df_S = df[df['Season']=='Summer']
# draw the boxplots for the Summer Olympics
draw_group(df_S, "Athlets per Sport (Summer Olympics)")

Ahora usemos la misma función definida anteriormente para trazar los deportes en los Juegos Olímpicos de Invierno.

In [25]:
# select only Winter Olympics
df_W = df[df['Season']=='Winter']
# draw the boxplots for the Summer Olympics
draw_group(df_W, "Athlets per Sport (Winter Olympics)",600)

### Heatmap

También usemos un 'Heatmap' para mostrar el número de atletas por evento de juego y por deporte.

Procesaremos aquí solo los datos de los Juegos Olímpicos de verano.

Primero creamos una matriz con las filas `Year` y las columnas` Sport` que tienen los valores del número de atletas por año y deporte.

In [26]:
piv = pd.pivot_table(df_S, values="Athlets",index=["Year"], columns=["Sport"], fill_value=0)
m = piv.values

In [27]:
piv.head(5)

Sport,Aeronautics,Alpinism,Archery,Art Competitions,Athletics,Badminton,Baseball,Basketball,Basque Pelota,Beach Volleyball,...,Table Tennis,Taekwondo,Tennis,Trampolining,Triathlon,Tug-Of-War,Volleyball,Water Polo,Weightlifting,Wrestling
Year,,,,,,,,,,,,,,,,,,,,,
1896,0,0,0,0,106,0,0,0,0,0,...,0,0,23,0,0,0,0,0,10,5
1900,0,0,32,0,234,0,0,0,2,0,...,0,0,47,0,0,12,0,53,0,0
1904,0,0,70,0,201,0,0,0,0,0,...,0,0,57,0,0,30,0,21,7,48
1906,0,0,0,0,470,0,0,0,0,0,...,0,0,48,0,0,32,0,0,22,39
1908,0,0,77,0,778,0,0,0,0,0,...,0,0,84,0,0,40,0,28,0,133


Los atributos que usamos son:
* z - the matrix with values to be displayed;
* x - the columns names;
* y - the rows names;  
* colorsacale - the color scale to be used for display; 

In [28]:
trace = go.Heatmap(z = m, y= list(piv.index), x=list(piv.columns),colorscale='Reds',reversescale=False)
data=[trace]
layout = dict(title = "Number of athlets per year and sport (Summer Olympics)",
              xaxis = dict(title = 'Sport',
                        showticklabels=True,
                           tickangle = 45,
                        tickfont=dict(
                                size=10,
                                color='black'),
                          ),
              yaxis = dict(title = 'Year', 
                        showticklabels=True, 
                        tickfont=dict(
                            size=10,
                            color='black'),
                      ), 
              hovermode = 'closest',
              showlegend=False,
                  width=1000,
                  height=800,
             )
fig = dict(data=data, layout=layout)
iplot(fig, filename='labelled-heatmap')

Mostremos también el diagrama de mapa de calor correspondiente para los Juegos Olímpicos de Invierno.

In [29]:
piv = pd.pivot_table(df_W, values="Athlets",index=["Year"], columns=["Sport"], fill_value=0)
m = piv.values

In [30]:
trace = go.Heatmap(z = m, y= list(piv.index), x=list(piv.columns),colorscale='Blues',reversescale=False)
data=[trace]
layout = dict(title = "Number of athlets per year and sport (Winter Olympics)",
              xaxis = dict(title = 'Sport',
                        showticklabels=True,
                           tickangle = 30,
                        tickfont=dict(
                                size=8,
                                color='black'),
                          ),
              yaxis = dict(title = 'Year', 
                        showticklabels=True, 
                        tickfont=dict(
                            size=10,
                            color='black'),
                      ), 
              hovermode = 'closest',
              showlegend=False,
                  width=800,
                  height=800,
             )
fig = dict(data=data, layout=layout)
iplot(fig, filename='labelled-heatmap')

## Países

Primero fusionemos el `noc_regions_df` con el conjunto de datos `athlete_events_df`.

In [3]:
olympics_df = athlete_events_df.merge(noc_regions_df)

In [32]:
print("All Olympics data -  rows:",olympics_df.shape[0]," columns:", olympics_df.shape[1])

All Olympics data -  rows: 270767  columns: 17


In [33]:
olympics_df.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,602,Abudoureheman,M,22.0,182.0,75.0,China,CHN,2000 Summer,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,NaN,China,NaN


Primero, cambiemos el nombre de la columna `region` por` Country`.

Luego, vamos a mostrar cuántas ediciones fueron por cada país diferente.

### Choropleth


Usaremos para representar al País una representación `Choropleth`.

Especificamos los siguientes atributos: 
* locations - these are the countries;  
* locationmode - the mode used for specifying the locations; in our case, we will use the `country names` option; other options are `ISO-3` or `USA-states`. 
* z - the value displayed; 
* text - the text shown in the popup on hover;  
* colorscale - the color scale used for the areas on the map; 
* marker - the marker used for the border of map areas specified in locations;

Estos son solo una parte de los atributos importantes para un mapa `Choropleth`,

Para el diseño, hay un atributo `geo` con los siguientes parámetros:
* showframe - if a frame is drawn around the map;  
* showcoastlines - if coast lines are drawn around the continents; if set to `False` no continents are shown;  
* showlakes - if interior non-continental areas are shown;  
* projection - there are multiple options, most used being `Mercator`, `orthographic`, `natural earth` and `albers usa` for US counties.

In [4]:
olympics_df=olympics_df.rename(columns = {'region':'Country'})

In [35]:
tmp = olympics_df.groupby(['Country'])['Year'].nunique()
df = pd.DataFrame(data={'Editions': tmp.values}, index=tmp.index).reset_index()
df.head(2)

,Country,Editions
0,Afghanistan,14
1,Albania,11


In [36]:
trace = go.Choropleth(
            locations = df['Country'],
            locationmode='country names',
            z = df['Editions'],
            text = df['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'rainbow',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Editions',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Olympic countries',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

Mostremos por separado el número de eventos por país para los eventos de verano e invierno. Primero extraeremos una función.

In [37]:
tmp = olympics_df.groupby(['Country', 'Season'])['Year'].nunique()
df = pd.DataFrame(data={'Editions': tmp.values}, index=tmp.index).reset_index()
df.head(2)

,Country,Season,Editions
0,Afghanistan,Summer,14
1,Albania,Summer,8


In [38]:
dfS = df[df['Season']=='Summer']; dfW = df[df['Season']=='Winter']

def draw_map(dataset, title, colorscale, reversescale=False):
    trace = go.Choropleth(
                locations = dataset['Country'],
                locationmode='country names',
                z = dataset['Editions'],
                text = dataset['Country'],
                autocolorscale =False,
                reversescale = reversescale,
                colorscale = colorscale,
                marker = dict(
                    line = dict(
                        color = 'rgb(0,0,0)',
                        width = 0.5)
                ),
                colorbar = dict(
                    title = 'Editions',
                    tickprefix = '')
            )

    data = [trace]
    layout = go.Layout(
        title = title,
        geo = dict(
            showframe = True,
            showlakes = False,
            showcoastlines = True,
            projection = dict(
                type = 'orthographic'
            )
        )
    )
    fig = dict( data=data, layout=layout )
    iplot(fig)
    
draw_map(dfS, 'Olympic countries (Summer games)', "Reds")

In [39]:
draw_map(dfW, 'Olympic countries (Winter games)', "Blues")

Muestremos la variación en el tiempo del número de atletas por cada país.

In [40]:
tmp = olympics_df.groupby(['Year','Sport'])['Country'].value_counts()
dataset = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()
dataset.head()

,Year,Sport,Country,Athlets
0,1896,Athletics,Greece,36
1,1896,Athletics,USA,21
2,1896,Athletics,Germany,14
3,1896,Athletics,France,12
4,1896,Athletics,UK,7


## Atletas

Vamos a mostrar la distribución de la edad, la altura y el peso de los atletas usando una tabla de 'distplot'.

Agruparemos los datos por sexo y temporada.

### create_distplot

Vamos a mostrar primero la distribución de la altura de los atletas, agrupados por sexo.

In [41]:
female_h = olympics_df[olympics_df['Sex']=='F']['Height'].dropna()
male_h = olympics_df[olympics_df['Sex']=='M']['Height'].dropna()

hist_data = [female_h, male_h]
group_labels = ['Female Height', 'Male Height']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Athlets Height distribution plot')
iplot(fig, filename='dist_only')

Vamos a mostrar la distribución del peso de los atletas, agrupados por sexo.

In [42]:
female_w = olympics_df[olympics_df['Sex']=='F']['Weight'].dropna()
male_w = olympics_df[olympics_df['Sex']=='M']['Weight'].dropna()

hist_data = [female_w, male_w]
group_labels = ['Female Weight', 'Male Weight']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Athlets Weight distribution plot')
iplot(fig, filename='dist_only')

Mostremos también la distribución por edades de los atletas, agrupados por sexo.

In [43]:
female_a = olympics_df[olympics_df['Sex']=='F']['Age'].dropna()
male_a = olympics_df[olympics_df['Sex']=='M']['Age'].dropna()

hist_data = [female_a, male_a]
group_labels = ['Female Age', 'Male Age']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Athlets Age distribution plot')
iplot(fig, filename='dist_only')

Mostremos en un gráfico con el eje `x` la altura promedio y con el eje `y` el peso promedio el número de atletas, agrupados por deporte.

Usaremos un diagrama de dispersión (scatter) pero con marcadores (para cada deporte) 

In [5]:
tmp = olympics_df.groupby(['Sport'])['Height', 'Weight'].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sport'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)
dataset.head(5)

C:\Users\PERSONAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Sport,Height,Weight,ID
0,Alpine Skiing,173.489052,72.068110,8829
1,Archery,173.203085,70.011135,2334
2,Art Competitions,174.644068,75.290909,3578
3,Athletics,176.258719,69.253138,38596
4,Badminton,174.235380,68.275130,1436


Definamos el texto emergente

In [10]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Sport: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Sport'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

Ahora creemos el diagrama de dispersión de burbujas.

In [46]:
data = []
for sport in dataset['Sport']:
    ds = dataset[dataset['Sport']==sport]
    trace = go.Scatter(
        x = ds['Height'],
        y = ds['Weight'],
        name = sport,
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=10,
            size=ds['ID'],
            line=dict(
                width=2
            ),),
        text = ds['hover_text']
    )
    data.append(trace)
                         
layout = go.Layout(
    title='Athlets height and weight mean - grouped by sport',
    xaxis=dict(
        title='Height [cm]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    yaxis=dict(
        title='Weight [kg]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(254, 254, 254)',
    showlegend=False,
)


fig = dict(data = data, layout = layout)

iplot(fig, filename='athlets_body_measures')
                         

### Slider (animation)

Representemos el diagrama de medidas del cuerpo de los atletas, agrupados no solo por `Sport` sino también por`Year`. Para cada `Año` estableceremos una diapositiva y la evolución en el tiempo se mostrará como una animación. También se agregará un botón de inicio y uno de pausa al gráfico, para permitir una fácil navegación a través de los cuadros.


In [47]:
tmp = olympics_df.groupby(['Sport', 'Year'])['Height', 'Weight'].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sport', 'Year'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)

C:\Users\PERSONAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [48]:
dataset.head(3)

,Sport,Year,Height,Weight,ID
0,Alpine Skiing,1936,169.250000,61.000000,103
1,Alpine Skiing,1948,170.116279,64.666667,360
2,Alpine Skiing,1952,173.387755,66.936170,378


Creamos el texto emergente, agregando también el `Año` en este caso.

In [49]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Year: {}<br>'+
                       'Sport: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Year'], 
                                            row['Sport'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

Creamos ahora la animación.

In [50]:
years = (olympics_df.groupby(['Year'])['Year'].nunique()).index
sports = (olympics_df.groupby(['Sport'])['Sport'].nunique()).index
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [140, 200], 'title': 'Height'}
figure['layout']['yaxis'] = {'range': [20, 200],'title': 'Weight'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['showlegend'] = False
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1896',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
# make data
year = 1896
for sport in sports:
    dataset_by_year = dataset[dataset['Year'] == year]
    dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sport'] == sport]

    data_dict = {
        'x': list(dataset_by_year_and_season['Height']),
        'y': list(dataset_by_year_and_season['Weight']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_season['hover_text']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 1,
            'size': list(dataset_by_year_and_season['ID'])
        },
        'name': sport
    }
    figure['data'].append(data_dict)
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for sport in sports:
        dataset_by_year = dataset[dataset['Year'] == int(year)]
        dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sport'] == sport]

        data_dict = {
            'x': list(dataset_by_year_and_season['Height']),
            'y': list(dataset_by_year_and_season['Weight']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_season['hover_text']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'size':  list(dataset_by_year_and_season['ID'])
            },
            'name': sport
        }
        frame['data'].append(data_dict)
    
    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
figure['layout']['sliders'] = [sliders_dict]
iplot(figure)

### Athlets body measurements grouped by Sex

Hagamoslo ahora agrupando los atletas por "Sexo" en lugar de "Deporte".

In [51]:
tmp = olympics_df.groupby(['Sex'])['Height', 'Weight'].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sex'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)
dataset.head()

C:\Users\PERSONAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Sex,Height,Weight,ID
0,F,167.845593,60.025894,74393
1,M,178.861121,75.748537,196374


In [52]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Sex: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Sex'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [53]:
data = []
for sex in dataset['Sex']:
    ds = dataset[dataset['Sex']==sex]
    trace = go.Scatter(
        x = ds['Height'],
        y = ds['Weight'],
        name = sex,
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=10,
            size=ds['ID'],
            line=dict(
                width=2
            ),),
        text = ds['hover_text']
    )
    data.append(trace)
                         
layout = go.Layout(
    title='Athlets height and weight mean - grouped by Sex',
    xaxis=dict(
        title='Height [cm]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    yaxis=dict(
        title='Weight [kg]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(254, 254, 254)',
    showlegend=False,
)


fig = dict(data = data, layout = layout)

iplot(fig, filename='athlets_body_measures2')
                         

### Variación temporal de la medición corporal de los atletas, agrupada por sexo.

In [54]:
tmp = olympics_df.groupby(['Sex', 'Year'])['Height', 'Weight'].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sex', 'Year'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)

C:\Users\PERSONAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [55]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Year: {}<br>'+
                       'Sex: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Year'], 
                                            row['Sex'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [56]:
years = (olympics_df.groupby(['Year'])['Year'].nunique()).index
sexes = (olympics_df.groupby(['Sex'])['Sex'].nunique()).index
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [100, 200], 'title': 'Height'}
figure['layout']['yaxis'] = {'range': [20, 200],'title': 'Weight'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['showlegend'] = False
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1896',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
# make data
year = 1896
for sex in sexes:
    dataset_by_year = dataset[dataset['Year'] == year]
    dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sex'] == sex]

    data_dict = {
        'x': list(dataset_by_year_and_season['Height']),
        'y': list(dataset_by_year_and_season['Weight']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_season['hover_text']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 1,
            'size': list(dataset_by_year_and_season['ID'])
        },
        'name': sex
    }
    figure['data'].append(data_dict)
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for sex in sexes:
        dataset_by_year = dataset[dataset['Year'] == int(year)]
        dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sex'] == sex]

        data_dict = {
            'x': list(dataset_by_year_and_season['Height']),
            'y': list(dataset_by_year_and_season['Weight']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_season['hover_text']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'size':  list(dataset_by_year_and_season['ID'])
            },
            'name': sex
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
figure['layout']['sliders'] = [sliders_dict]
iplot(figure)

### Mediciones corporales de atletas, agrupadas por sexo y deporte.

Agrupemos ahora en ambos criterios y creemos dos gráficos. Agregamos también la 'Edad' promedio y hacemos que el tamaño de la burbuja sea proporcional a la Edad.

In [57]:
tmp = olympics_df.groupby(['Sport', 'Sex'])['Height', 'Weight', 'Age'].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sport', 'Sex'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)

C:\Users\PERSONAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [58]:
dataset.head()

,Sport,Sex,Height,Weight,Age,ID
0,Alpine Skiing,F,167.221001,62.640307,22.334609,3398
1,Alpine Skiing,M,177.891374,78.626035,23.758266,5431
2,Archery,F,167.166483,62.013575,26.508458,1015
3,Archery,M,178.477842,77.066866,29.083267,1319
4,Art Competitions,M,174.896552,75.290909,46.062816,3201


In [59]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Sex: {}<br>'+
                       'Sport: {}<br>'
                       'Number of athlets: {}<br>'+
                       'Mean Age: {}<br>'
                       'Mean Height: {}<br>'+
                       'Mean Weight: {}<br>').format(row['Sex'],
                                            row['Sport'],
                                            row['ID'],
                                            round(row['Age'],2), 
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [60]:

def plot_bubble_chart(dataset,title):
    data = []
    for sport in dataset['Sport']:
        ds = dataset[dataset['Sport']==sport]
        trace = go.Scatter(
            x = ds['Height'],
            y = ds['Weight'],
            name = sport,
            marker=dict(
                symbol='circle',
                sizemode='area',
                sizeref=50,
                size=np.power(ds['Age'],3),
                line=dict(
                    width=2
                ),),
            text = ds['hover_text']
        )
        data.append(trace)

    layout = go.Layout(
        title= title,
        xaxis=dict(
            title='Height [cm]',
            gridcolor='rgb(128, 128, 128)',
            zerolinewidth=1,
            ticklen=1,
            gridwidth=0.5,
            range=[150,200]
        ),
        yaxis=dict(
            title='Weight [kg]',
            gridcolor='rgb(128, 128, 128)',
            zerolinewidth=1,
            ticklen=1,
            gridwidth=0.5,
            range=[45,100]
        ),
        paper_bgcolor='rgb(255,255,255)',
        plot_bgcolor='rgb(254, 254, 254)',
        showlegend=False,
    )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='athlets_body_measures')
    


In [61]:
dF = dataset[dataset['Sex']=='F']
plot_bubble_chart(dF,'Female athlets height and weight mean - grouped by sport')

In [66]:
dM = dataset[dataset['Sex']=='M']
plot_bubble_chart(dM,'Male athlets height and weight mean - grouped by sport')

## Medallas 

Veamos cuáles son los países con más medallas.

In [62]:
tmp = olympics_df.groupby(['Country', 'Medal'])['ID'].agg('count').dropna()
df = pd.DataFrame(tmp).reset_index()

In [63]:
dfG = df[df['Medal']=='Gold']
dfS = df[df['Medal']=='Silver']
dfB = df[df['Medal']=='Bronze']

def draw_map(dataset, title, colorscale):
    trace = go.Choropleth(
                locations = dataset['Country'],
                locationmode='country names',
                z = dataset['ID'],
                text = dataset['Country'],
                autocolorscale =False,
                colorscale = colorscale,
                marker = dict(
                    line = dict(
                        color = 'rgb(0,0,0)',
                        width = 0.5)
                ),
                colorbar = dict(
                    title = 'Medals',
                    tickprefix = '')
            )
    data = [trace]
    layout = go.Layout(
        title = title,
        geo = dict(
            showframe = True,
            showlakes = False,
            showcoastlines = True,
            projection = dict(
                type = 'natural earth'
            )
        )
    )
    fig = dict( data=data, layout=layout )
    iplot(fig)

Tracemos los países con medallas de oro, plata y bronce.

In [64]:
draw_map(dfG, "Countries with Gold Medals",'Greens')

In [65]:
draw_map(dfS, "Countries with Silver Medals",'Greys')

In [66]:
draw_map(dfB, "Countries with Bronze Medals",'Reds')

Vamos a mostrar el número de medallas (oro, plata, bronce) por edición olímpica.

In [67]:
tmp = olympics_df.groupby(['Year', 'City','Season', 'Medal'])['ID'].agg('count').dropna()
df = pd.DataFrame(tmp).reset_index()
dfG = df[df['Medal']=='Gold']
dfS = df[df['Medal']=='Silver']
dfB = df[df['Medal']=='Bronze']

In [68]:
dfG.head()

,Year,City,Season,Medal,ID
1,1896,Athina,Summer,Gold,62
4,1900,Paris,Summer,Gold,201
7,1904,St. Louis,Summer,Gold,173
10,1906,Athina,Summer,Gold,157
13,1908,London,Summer,Gold,294


In [69]:

traceG = go.Bar(
    x = dfG['Year'],y = dfG['ID'],
    name="Gold",
     marker=dict(
                color='gold',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text = dfG['City']+ " (" + dfG['Season'] + ")",
)
traceS = go.Bar(
    x = dfS['Year'],y = dfS['ID'],
    name="Silver",
    marker=dict(
                color='Grey',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfS['City']+ " (" + dfS['Season'] + ")",
)

traceB = go.Bar(
    x = dfB['Year'],y = dfB['ID'],
    name="Bronze",
    marker=dict(
                color='Brown',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfB['City']+ " (" + dfB['Season'] + ")",
)

data = [traceG, traceS, traceB]
layout = dict(title = 'Medals per Olympic edition',
          xaxis = dict(title = 'Year', showticklabels=True), 
          yaxis = dict(title = 'Number of medals'),
          hovermode = 'closest',
          barmode='stack'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')

También vamos a mostrar el número de medallas por deporte.
Mostraremos por separado el número de medallas para **Oro**, **Plata** y **Bronce**.

In [70]:
tmp = olympics_df.groupby(['Sport', 'Medal'])['ID'].agg('count').dropna()
df = pd.DataFrame(tmp).reset_index()
dfG = df[df['Medal']=='Gold']
dfS = df[df['Medal']=='Silver']
dfB = df[df['Medal']=='Bronze']

In [71]:
traceG = go.Bar(
    x = dfG['Sport'],y = dfG['ID'],
    name="Gold",
     marker=dict(
                color='gold',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text = dfG['Sport'],
    #orientation = 'h'
)
traceS = go.Bar(
    x = dfS['Sport'],y = dfS['ID'],
    name="Silver",
    marker=dict(
                color='Grey',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfS['Sport'],
    #orientation = 'h'
)

traceB = go.Bar(
    x = dfB['Sport'],y = dfB['ID'],
    name="Bronze",
    marker=dict(
                color='Brown',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfB['Sport'],
   # orientation = 'h'
)

data = [traceG, traceS, traceB]
layout = dict(title = 'Medals per sport',
          xaxis = dict(title = 'Sport', showticklabels=True, tickangle=45,
            tickfont=dict(
                size=8,
                color='black'),), 
          yaxis = dict(title = 'Number of medals'),
          hovermode = 'closest',
          barmode='stack',
          showlegend=False,
          width=900,
          height=600,
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')